<a href="https://colab.research.google.com/github/omar1slam/Random-ML-Scripts/blob/master/GA%20%26%20PSO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!pip install --upgrade gym
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

Requirement already up-to-date: gym in /usr/local/lib/python3.6/dist-packages (0.15.4)


In [ ]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
!tf_upgrade_v2 -h

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (43.0.0)
usage: tf_upgrade_v2 [-h] [--infile INPUT_FILE] [--outfile OUTPUT_FILE]
                     [--intree INPUT_TREE] [--outtree OUTPUT_TREE]
                     [--copyotherfiles COPY_OTHER_FILES] [--inplace]
                     [--reportfile REPORT_FILENAME] [--mode {DEFAULT,SAFETY}]
                     [--print_all]

Convert a TensorFlow Python file from 1.x to 2.0

Simple usage:
  tf_upgrade_v2.py --infile foo.py --outfile bar.py
  tf_upgrade_v2.py --infile foo.ipynb --outfile bar.ipynb
  tf_upgrade_v2.py --intree ~/code/old --outtree ~/code/new

optional arguments:
  -h, --help            show this help message and exit
  --infile INPUT_FILE   If converting a single file, the name of the file to
                        convert
  --outfile OUTPUT_FILE
                        If converting a single file, the output filename.
  --intree INPUT_TREE   If converting a whole tree of files, the d

In [ ]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
import time, math, random, bisect, copy

from ast import literal_eval

from IPython import display as ipythondisplay

In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [ ]:
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [ ]:
env = wrap_env(gym.make('CartPole-v1'))

In [ ]:
print(env.action_space)
print(env.observation_space)

Discrete(2)
Box(4,)


In [ ]:
def sigmoid(x):
  return 1/(1 + np.exp(-x)) 

class NeuralNet : 
    def __init__(self, arch): 
        '''
        in order to initialize the neural network class we need to specify the 
        number of layers and nodes in each layer
        For each neural network there is a fitness in order to evaluate such network
        and initialization of the weights/bias within that 
        '''
        self.fitness = 0.0
        self.arch = arch
        self.weights = []
        self.biases = []
        for i in range(len(arch) - 1):
            self.weights.append( np.random.uniform(low=-1, high=1, size=(arch[i], arch[i+1])))
            self.biases.append( np.random.uniform(low=-1, high=1, size=(arch[i+1])))

    def printWeightsandBiases(self):
          
        print("--------------------------------")
        print("Weights :\n[", end="")
        for i in range(len(self.weights)):
            print("\n [ ", end="")
            for j in range(self.weights[i].shape[0]):
                if j!=0:
                    print("\n   ", end="")
                print("[", end="")
                for k in range(self.weights[i][j].shape[0]):
                    print(" %5.2f," % (self.weights[i][j,k]), end="")
                print("\b],", end="")
            print("\b ],")
        print("\n]")

        print("\nBiases :\n[", end="")
        for i in range(len(self.biases)):
            print("\n [ ", end="")
            for j in range(self.biases[i].shape[0]):
                    print(" %5.2f," % (self.biases[i][j]), end="")
            print("\b],", end="")
        print("\b \n]\n--------------------------------\n")
    
    
    def getOutput(self, input):
        output = input
        for i in range(len(self.arch)-1):
          #remove the reshape
            output = np.reshape( np.dot(output, self.weights[i]) + self.biases[i], (self.arch[i+1],))
            output = np.tanh(output)
        output = sigmoid(output)
        output = np.argmax(output)
        return output

In [ ]:
class Population :
    def __init__(self, populationCount, mutationRate, arch):
        self.arch = arch
        self.popCount = populationCount
        self.m_rate = mutationRate
        self.population = [ NeuralNet(arch) for i in range(populationCount)]


    def createChild(self, nn1, nn2):
        child = NeuralNet(self.arch)
        for i in range(len(child.weights)):
            for j in range(child.weights[i].shape[0]):
                for k in range(child.weights[i][j].shape[0]):
                    if random.random() < nn1.fitness / (nn1.fitness+nn2.fitness):
                        child.weights[i][j,k] = nn1.weights[i][j,k] 
                    else :
                        child.weights[i][j,k] = nn2.weights[i][j,k]
                    if random.random() < self.m_rate:
                        child.weights[i][j][k] = child.weights[i][j,k] + 0.005*(2*np.random.rand()-1)

        for i in range(len(child.biases)):
            for j in range(child.biases[i].shape[0]):
                if random.random() < nn1.fitness / (nn1.fitness+nn2.fitness):
                    child.biases[i][j] = nn1.biases[i][j]
                else:
                    child.biases[i][j] = nn2.biases[i][j]
                if random.random() < self.m_rate:
                    child.biases[i][j] = nn2.biases[i][j] + 0.005*(2*np.random.rand()-1)
        return child


    def createNewGeneration(self, bestNN):    
        nextGen = []
        self.population.sort(key=lambda x: x.fitness, reverse=True)
        for i in range(self.popCount):
            if random.random() < float(self.popCount-i)/self.popCount:
                nextGen.append(copy.deepcopy(self.population[i]));

        fitnessSum = [0]
        minFit = min([i.fitness for i in nextGen])
        for i in range(len(nextGen)):
            fitnessSum.append(fitnessSum[i]+(nextGen[i].fitness-minFit))
        

        while(len(nextGen) < self.popCount):
            r1 = random.uniform(0, fitnessSum[len(fitnessSum)-1] )
            r2 = random.uniform(0, fitnessSum[len(fitnessSum)-1] )
            i1 = bisect.bisect_left(fitnessSum, r1)
            i2 = bisect.bisect_left(fitnessSum, r2)
            if 0 <= i1 < len(nextGen) and 0 <= i2 < len(nextGen) :
                nextGen.append( self.createChild(nextGen[i1], nextGen[i2]) )
            else :
                print("Index Error ")
                print("Sum Array =",fitnessSum)
                print("Randoms = ", r1, r2)
                print("Indices = ", i1, i2)
        self.population.clear()
        self.population = nextGen

In [ ]:
MAX_STEPS = 500
MAX_GENERATIONS = 60
POPULATION_COUNT = 200
MUTATION_RATE = 0.01
observation = env.reset()
in_dimen = env.observation_space.shape[0]
out_dimen = env.action_space.n
pop = Population(POPULATION_COUNT, MUTATION_RATE, [in_dimen, 13, 8, 13, out_dimen])
bestNeuralNets = []

print("\nObservation\n--------------------------------")
print("Shape :", in_dimen)
print("\nAction\n--------------------------------")
print("Shape :", out_dimen)

for gen in range(MAX_GENERATIONS):
    genAvgFit = 0.0
    minFit =  1000000
    maxFit = -1000000
    maxNeuralNet = None
    for nn in pop.population:
        observation = env.reset()
        totalReward = 0
        for step in range(MAX_STEPS):
            action = nn.getOutput(observation)
            observation, reward, done, info = env.step(action)
            totalReward += reward
            if done:
                break

        nn.fitness = totalReward
        minFit = min(minFit, nn.fitness)
        genAvgFit += nn.fitness
        if nn.fitness > maxFit :
            maxFit = nn.fitness
            maxNeuralNet = copy.deepcopy(nn);

    bestNeuralNets.append(maxNeuralNet)
    genAvgFit/=pop.popCount
    print("Generation : {}  |  Min : {}  |  Avg : {}  |  Max : {}  ".format(gen+1, 
                                                                            np.around(minFit), 
                                                                            np.around(genAvgFit), 
                                                                            np.around(maxFit) ))
    pop.createNewGeneration(maxNeuralNet)




Observation
--------------------------------
Shape : 4

Action
--------------------------------
Shape : 2
Generation : 1  |  Min : 8.0  |  Avg : 17.0  |  Max : 126.0  
Generation : 2  |  Min : 8.0  |  Avg : 24.0  |  Max : 335.0  
Generation : 3  |  Min : 8.0  |  Avg : 26.0  |  Max : 162.0  
Generation : 4  |  Min : 8.0  |  Avg : 28.0  |  Max : 149.0  
Generation : 5  |  Min : 8.0  |  Avg : 37.0  |  Max : 248.0  
Generation : 6  |  Min : 8.0  |  Avg : 41.0  |  Max : 500.0  
Generation : 7  |  Min : 8.0  |  Avg : 52.0  |  Max : 500.0  
Generation : 8  |  Min : 8.0  |  Avg : 72.0  |  Max : 500.0  
Generation : 9  |  Min : 8.0  |  Avg : 114.0  |  Max : 500.0  
Generation : 10  |  Min : 8.0  |  Avg : 195.0  |  Max : 500.0  
Generation : 11  |  Min : 8.0  |  Avg : 291.0  |  Max : 500.0  
Generation : 12  |  Min : 8.0  |  Avg : 358.0  |  Max : 500.0  
Generation : 13  |  Min : 10.0  |  Avg : 386.0  |  Max : 500.0  
Generation : 14  |  Min : 16.0  |  Avg : 384.0  |  Max : 500.0  
Generation :

In [ ]:
env = wrap_env(gym.make('CartPole-v1'))
bestNeuralNets.sort(key=lambda x: x.fitness, reverse=True)
observation = env.reset()
totalReward = 0
for step in range(MAX_STEPS):
    env.render()
    action = bestNeuralNets[0].getOutput(observation)
    observation, reward, done, info = env.step(action)
    totalReward += reward
    if done:
        observation = env.reset()
        break
print("Expected Fitness of %4d | Actual Fitness = %4d" % (bestNeuralNets[0].fitness, totalReward))
env.close()

Expected Fitness of  500 | Actual Fitness =  500


In [ ]:
show_video()

In [ ]:
print(np.shape(observation))

(4,)


# **PSO**

In [ ]:

!pip install pyswarms

In [ ]:
import pyswarms as ps
%load_ext autoreload
%autoreload 2
print(pop.population)
env = wrap_env(gym.make('CartPole-v1'))

2020-01-01 15:44:59,227 - numexpr.utils - INFO - NumExpr defaulting to 2 threads.


[<__main__.NeuralNet object at 0x7f062e9e01d0>, <__main__.NeuralNet object at 0x7f062e9e02b0>, <__main__.NeuralNet object at 0x7f062e9e06d8>, <__main__.NeuralNet object at 0x7f062e9e0630>, <__main__.NeuralNet object at 0x7f062e9e0128>, <__main__.NeuralNet object at 0x7f062e9e0470>, <__main__.NeuralNet object at 0x7f062e9e0860>, <__main__.NeuralNet object at 0x7f062e9e0400>, <__main__.NeuralNet object at 0x7f062e9e0240>, <__main__.NeuralNet object at 0x7f062e9e0048>, <__main__.NeuralNet object at 0x7f062e9e04e0>, <__main__.NeuralNet object at 0x7f062e9e0358>, <__main__.NeuralNet object at 0x7f062e9e0f60>, <__main__.NeuralNet object at 0x7f062e9e0208>, <__main__.NeuralNet object at 0x7f062e9e0fd0>, <__main__.NeuralNet object at 0x7f062e9e0828>, <__main__.NeuralNet object at 0x7f062e9e03c8>, <__main__.NeuralNet object at 0x7f062e9e0da0>, <__main__.NeuralNet object at 0x7f062e9e06a0>, <__main__.NeuralNet object at 0x7f062e9e08d0>, <__main__.NeuralNet object at 0x7f062e9e0080>, <__main__.Ne

In [ ]:
def forward_prop(params,obsv):
    
    # Neural network architecture
    in_dimen = env.observation_space.shape[0]
    n_inputs = in_dimen
    n_hidden1 = 13
    n_hidden2 = 8
    n_hidden3 = 13 
    n_classes = env.action_space.n
    X=obsv
    

    # Roll-back the weights and biases
    #W1 = params[0:(in_dimen*n_hidden1)].reshape((n_inputs,n_hidden1))
    #b1 = params[(in_dimen*n_hidden1):(in_dimen*n_hidden1)+n_hidden1].reshape((n_hidden1,))
    #W2 = params[(in_dimen*n_hidden1)+n_hidden1:(in_dimen*n_hidden1)+n_hidden1+(n_hidden1*n_hidden2)].reshape((n_hidden1,n_hidden2))
    #b2 = params[(in_dimen*n_hidden1)+n_hidden1+(n_hidden1*n_hidden2):(in_dimen*n_hidden1)+n_hidden1+(n_hidden1*n_hidden2)+n_hidden2].reshape((n_hidden2,))
    #W3 = params[(in_dimen*n_hidden1)+n_hidden1+(n_hidden1*n_hidden2)+n_hidden2:(in_dimen*n_hidden1)+n_hidden1+(n_hidden1*n_hidden2)+n_hidden2+(n_hidden2*n_hidden3)].reshape((n_hidden2,n_hidden3))
    #b3 = params[(in_dimen*n_hidden1)+n_hidden1+(n_hidden1*n_hidden2)+n_hidden2+(n_hidden2*n_hidden3):(in_dimen*n_hidden1)+n_hidden1+(n_hidden1*n_hidden2)+n_hidden2+(n_hidden2*n_hidden3)+n_hidden3].reshape((n_hidden3,))
    #W4 = params[(in_dimen*n_hidden1)+n_hidden1+(n_hidden1*n_hidden2)+n_hidden2+(n_hidden2*n_hidden3)+n_hidden3:(in_dimen*n_hidden1)+n_hidden1+(n_hidden1*n_hidden2)+n_hidden2+(n_hidden2*n_hidden3)+n_hidden3+(n_hidden3*n_classes)].reshape((n_hidden3,n_classes))
    #b4 = params[(in_dimen*n_hidden1)+n_hidden1+(n_hidden1*n_hidden2)+n_hidden2+(n_hidden2*n_hidden3)+n_hidden3+(n_hidden3*n_classes):((in_dimen*n_hidden1)+n_hidden1+(n_hidden1*n_hidden2)+n_hidden2+(n_hidden2*n_hidden3)+n_hidden3+(n_hidden3*n_classes))+n_classes].reshape((n_classes,))
    W1 = params[0:52].reshape((in_dimen,n_hidden1))
    b1 = params[52:65].reshape((n_hidden1,))
    W2 = params[65:169].reshape((n_hidden1,n_hidden2))
    b2 = params[169:177].reshape((n_hidden2,))
    W3 = params[177:281].reshape((n_hidden2,n_hidden3))
    b3 = params[281:294].reshape((n_hidden3,))
    W4=params[294:320].reshape((n_hidden3,n_classes))
    b4 = params[320:322].reshape((2,))

    # Perform forward propagation
    z1 = X.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    z2 = a1.dot(W2) + b2 # Pre-activation in Layer 2
    a2 = np.tanh(z2)
    z3 = a2.dot(W3) + b3
    a3 = np.tanh(z3)
    z4 = a3.dot(W4) + b4
    logits = z4          # Logits for Layer 2

    # Compute for the softmax of the logits
    #exp_scores = np.exp(logits)
    #probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Compute for the negative log likelihood
    logits = sigmoid(logits)
    logits = np.argmax(logits)
    #print(1)
    

    return logits
 
        



In [ ]:
def f(x):
    
    n_particles = x.shape[0]
    env.reset()
    j = np.zeros((n_particles,1))
    observation = np.zeros((4,))
    for i in range(n_particles):
      totalReward=0
      env.reset()
      for k in range(MAX_STEPS):
        action = forward_prop(x[i], observation)
        observation,reward,done,info = env.step(action)
        totalReward+= reward
        if done:
          break
      j[i,0] = -totalReward
    j = j.reshape(n_particles,)
    return j


In [ ]:
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}
dimensions = (in_dimen*13)+(13*8)+(8*13)+(13*out_dimen)+13+13+8+out_dimen
optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=dimensions, options=options)

In [ ]:
env.reset()
cost, pos = optimizer.optimize(f, iters=100)

2020-01-01 15:45:05,885 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|100/100, best_cost=-500
2020-01-01 15:46:23,075 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -500.0, best pos: [ 1.13238307e+00 -1.24401732e+00  2.01314864e-01  8.83315805e-01
  1.78250752e+00 -3.39604268e-01  1.14702785e+00 -4.25222344e-02
  1.61743030e-01 -5.46544956e-01 -2.52588249e-01 -3.70586123e-02
  1.07240384e+00 -6.84269294e-01 -3.06644073e-01  9.82261755e-01
 -6.66657107e-01  5.93789677e-01  3.69095420e-01 -2.94167096e-01
  7.70459031e-01  1.04737334e+00  1.18303231e+00  1.45765271e+00
  1.10570491e+00 -9.31485820e-01 -5.16900845e-01  1.12881660e+00
 -8.85037412e-01  2.03991103e-01  1.34520642e+00  3.29093219e-01
  1.90277521e+00 -6.13514436e-01  3.32049933e-01  1.23520427e+00
  1.85031395e-01  5.50605161e-01  5.42462481e-01 -7.84258288e-02
  2.26154627e+00  1.49185792e+00  1.05703

In [ ]:
env = wrap_env(gym.make('CartPole-v1'))
observation = env.reset()
for step in range(MAX_STEPS):
    env.render()
    action = forward_prop(pos,observation)
    observation, reward, done, info = env.step(action)
    if done:
        break

In [ ]:
show_video()

# Easy Function using GA

In [ ]:
class population:

    def __init__(self, f, pop_size = 100, n_variables = 2):
        self.f = f
        self.minim = -100
        self.maxim = 100
        self.pop_size = pop_size
        self.n_variables = n_variables
        self.population = self.init()
        self.GetPop()

    def init(self):
        return [np.random.randint(self.minim, self.maxim, size=(self.n_variables)) 
                           for i in range(self.pop_size)]

    def GetPop(self):
        return [self.f(i[0], i[1]) for i in self.population]
        #return [(i[0]-4)**2 + i[1]**2 for i in self.population]

    def nextGen(self):
        results = self.GetPop()
        children = [self.population[np.argmin(results)]]

        while len(children) < self.pop_size:
            randA, randB = np.random.randint(0, self.pop_size), \
                           np.random.randint(0, self.pop_size)
            if results[randA] < results[randB]: p1 = self.population[randA]
            else: p1 = self.population[randB]

            randA, randB = np.random.randint(0, self.pop_size), \
                           np.random.randint(0, self.pop_size)  
            if results[randA] < results[randB]: p2 = self.population[randA]
            else: p2 = self.population[randB]   

            signs = []
            for i in zip(p1, p2):
                if i[0] < 0 and i[1] < 0: signs.append(-1)
                elif i[0] >= 0 and i[1] >= 0: signs.append(1)
                else: signs.append(np.random.choice([-1,1]))

            p1 = [format(abs(i), '010b') for i in p1]
            p2 = [format(abs(i), '010b') for i in p2]

            child = []
            for i, j in zip(p1, p2):
                for k, l in zip(i, j):
                    if k == l: child.append(k)
                    else: child.append(str(np.random.randint(min(k, l), 
                                                             max(k,l))))

            child = ''.join(child)
            g1 = child[0:len(child)//2] 
            g2 = child[len(child)//2:len(child)]
            children.append(np.asarray([signs[0]*int(g1, 2), 
                                        signs[1]*int(g2, 2)]))
        self.population = children

    def run(self):
        ix = 0
        while ix < 100:
            ix += 1
            self.nextGen()
        return self.population[0]

In [ ]:
fx = lambda x,y=0: x**2 + 2*x + 4
gen = Genetic(fx)
minim = gen.run()
print("Best x=%f | Minimum f(x)=%f" % (minim[0], fx(minim[0])))

Best x=-1.000000 | Minimum f(x)=3.000000


# Easy Function using PSO

In [ ]:
def F(x):
  return x**2 + 2*x + 4

def Y(y):
  n_particles = y.shape[0]
  j = [F(y[i][0]) for i in range(n_particles)]
  j = np.array(j)
  #j = j.reshape(n_particles,)
  return j


In [ ]:
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}
d = 1
optimizer = ps.single.GlobalBestPSO(n_particles=10, dimensions=d, options=options)
cost, pos = optimizer.optimize(Y, iters=100)

2020-01-01 19:54:20,565 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}










































pyswarms.single.global_best:   0%|          |0/100









































pyswarms.single.global_best:   0%|          |0/100, best_cost=4.07









































pyswarms.single.global_best:   0%|          |0/100, best_cost=4.07









































pyswarms.single.global_best:   0%|          |0/100, best_cost=4.07









































pyswarms.single.global_best:   0%|          |0/100, best_cost=4.07









































pyswarms.single.global_best:   0%|          |0/100, best_cost=3.95









































pyswarms.single.global_best:   0%|          |0/100, best_cost=3.28









































pyswarms.single.global_best:   0%|          |0/100, best_cost=3.01





























In [ ]:
print("Best x=%f | Minimum f(x)=%f" % (pos[0],F(pos[0])))

Best x=-0.999988 | Minimum f(x)=3.000000
